Let's go through SVD first.

The equation is:

$\hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u$


The loss function is

$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 + \lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right)$



The update rule

$b_u \leftarrow b_u + \gamma (e_{ui} - \lambda b_u)$

$b_i \leftarrow b_i + \gamma (e_{ui} - \lambda b_i)$

$p_u \leftarrow p_u + \gamma (e_{ui} \cdot q_i - \lambda p_u)$

$q_i \leftarrow q_i + \gamma (e_{ui} \cdot p_u - \lambda q_i)$

We assume the input X, consists of three columns: the user_id, the item_id, and the rating.

In [1]:
import numpy as np


class SVD:
    def __init__(self, learning_rate, regularized_rate, max_step, n_users, n_items, n_factors):
        self.learning_rate = learning_rate
        self.regularized_rate = regularized_rate
        self.max_step = max_step
        self.bu = np.zeros(n_users, np.double)
        self.bi = np.zeros(n_items, np.double)
        self.pu = np.zeros((n_users, n_factors), np.double)
        self.qi = np.zeros((n_items, n_factors), np.double)
        self.mean = 0
        
    def get_pred_value(u, i):
        return self.mean + self.bu[u] + self.bi[i] + np.dot(self.pu[u], self.qi[i])
        
    def fit(self, X, y):
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
            err = r - self.get_pred_value(u, i)
            self.bu[u] += self.learning_rate * (err - self.regularized_rate * self.bu[u])
            self.bi[i] += self.learning_rate * (err - self.regularized_rate * self.bi[i])
            tmp = self.pu[u]
            self.pu[u] += self.learning_rate * (err * self.qi[i] - self.regularized_rate * self.pu[u])
            self.qi[i] += self.learning_rate * (err * tmp - self.regularized_rate * self.qi[i])
            if index == self.max_step:
                break
        return self
            
    def transform(self, X):
        result = [0] * len(X)
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
            result[index] = self.get_pred_value(u, i)
        return result

Here comes SVD++.

$\hat{r}_{ui} = \mu + b_u + b_i + q_i^T\left(p_u + |I_u|^{-\frac{1}{2}} \sum_{j \in I_u}y_j\right)$

Remember we also modified the loss function:

$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 + \lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2 + ||y_j||^2 \right)$

And the update rule:

$q_i \leftarrow q_i + \gamma (e_{ui} \cdot (p_u + |I_u|^{-\frac{1}{2}} \sum_{j \in I_u}y_j) - \lambda q_i)$

In [2]:
class SVDPP:
    def __init__(self, learning_rate, regularized_rate, max_step, n_users, n_items, n_factors, ur_dict):
        self.learning_rate = learning_rate
        self.regularized_rate = regularized_rate
        self.max_step = max_step
        self.bu = np.zeros(n_users, np.double)
        self.bi = np.zeros(n_items, np.double)
        self.pu = np.zeros((n_users, n_factors), np.double)
        self.qi = np.zeros((n_items, n_factors), np.double)
        self.yj = np.zeros((n_items, n_factors), np.double)
        self.ur_dict = ur_dict
        self.mean = 0
        
    def get_pred_value(u, i, u_impl_fdb):
        return self.mean + self.bu[u] + self.bi[i] + np.dot(self.pu[u] + u_impl_fdb, self.qi[i])
        
    def fit(self, X, y):
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
            
            # suppose we can get a user's rating related to all items he rated
            Iu = [j for (j, _) in self.ur_dict[u].items( )]
            sqrt_Iu = np.sqrt(len(Iu))
            u_impl_fdb = np.zeros(self.n_factors, np.double)
            for j in Iu:
                u_impl_fdb += self.yj[j] / sqrt_Iu
            
            err = r - self.get_pred_value(u, i, u_impl_fdb)
            self.bu[u] += self.learning_rate * (err - self.regularized_rate * self.bu[u])
            self.bi[i] += self.learning_rate * (err - self.regularized_rate * self.bi[i])
            self.pu[u] += self.learning_rate * (err * self.qi[i] - self.regularized_rate * self.pu[u])
            self.qi[i] += self.learning_rate * (err * (self.pu[u] + u_impl_fdb) - self.regularized_rate * self.qi[i])
            
            for j in Iu:
                self.yj[j] += self.learning_rate * (err * self.qi[i] / sqrt_Iu - self.regularized_rate * self.yj[j])
                
            if index == self.max_step:
                break
        return self
            
    def transform(self, X):
        result = [0] * len(X)
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
            result[index] = self.get_pred_value(u, i)
        return result